In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time, os, cv2
import json
from ultralytics import YOLO

import sys
sys.path.append(os.path.abspath("/home/ros/llm_robot"))
from utils.pixel_to_cam_coords import detect_objects
from utils.camera import capture_d455_images, load_intrinsics

In [59]:
from pymycobot import MyCobot280

mc = MyCobot280("/dev/ttyACM1", 115200)

In [60]:
mc.get_servo_temps()

[28, 28, 28, 45, 34, 40]

In [64]:
mc.release_all_servos()
input("Press Enter to continue...")
mc.power_on()
mc.get_coords(), mc.get_angles() #2.9 -> 

([35.8, -62.6, 421.9, -88.32, -1.76, -88.9],
 [0.87, 2.98, -1.31, 0.0, 0.26, -1.75])

In [53]:
mc.send_coords([172,-19.43,200,-179,0,0],20,1)

1

In [65]:
print("Intel Real Sense 캡쳐 수행")
c_path, d_path, intr_path = capture_d455_images()
# coords = [81.6, -67.5, 299.4, -171.96, -3.71, -94.06]
print("Saved:", c_path, d_path)

Intel Real Sense 캡쳐 수행
Saved: /home/ros/llm_robot/data/captures/color/2025-08-27_12-27-14.jpg /home/ros/llm_robot/data/captures/depth/2025-08-27_12-27-14.npy


In [66]:
# RGB에서 YOLO로 바운딩 박스 얻기
best_model = "/home/ros/llm_robot/yolo/runs/pose/yolo11n_640_500ep/weights/best.pt"
target_list = ["cube"]
print("🎯 YOLO 대상 객체:", target_list)

# bbox: 바운딩 박스 모서리
# pixel_xy: 박스 중앙 픽셀
# cam_xyz: 카메라 좌표 (실제 3D위치)
camera_matrix, dist_coeffs = load_intrinsics(intr_path)

outputs = detect_objects(
    c_path=c_path,
    d_path=d_path,
    target_list=target_list,
    camera_matrix=camera_matrix,
    dist_coeffs=dist_coeffs,
    best_model=best_model
)

🎯 YOLO 대상 객체: ['cube']

image 1/1 /home/ros/llm_robot/data/captures/color/2025-08-27_12-27-14.jpg: 384x640 1 cube, 5.9ms
Speed: 0.9ms preprocess, 5.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

📦 추출된 객체 정보
cube        z=0.536 m  cam=(0.468, 0.074, 0.536)


In [34]:
import cv2
import numpy as np
from pathlib import Path
from typing import Tuple, List, Optional

def get_cam_xyz(x, y, depth_raw, camera_matrix, dist_coeffs, depth_scale=0.001):
    
    # 깊이값 가져오기 (ROI 주변 평균 사용)
    roi_size = 5
    y1, y2 = max(0, y - roi_size), min(depth_raw.shape[0], y + roi_size + 1)
    x1, x2 = max(0, x - roi_size), min(depth_raw.shape[1], x + roi_size + 1)
    
    roi = depth_raw[y1:y2, x1:x2]
    valid_depths = roi[roi > 0]
    
    if valid_depths.size == 0:
        print(f"[경고] 점 ({x}, {y})에서 유효한 깊이값을 찾을 수 없습니다.")
        clicked_point = None
        return None
    
    # 깊이값 계산 (중간값 사용)
    z_mm = np.median(valid_depths)
    z_m = z_mm * depth_scale
    
    # 카메라 내부 파라미터
    # fx = camera_matrix[0, 0]
    # fy = camera_matrix[1, 1]
    # ppx = camera_matrix[0, 2]
    # ppy = camera_matrix[1, 2]
    
    # 왜곡 보정
    pixel = np.array([[[x, y]]], dtype=np.float32)
    undistorted = cv2.undistortPoints(pixel, camera_matrix, dist_coeffs)
    x_u, y_u = undistorted[0][0]
    
    # 픽셀 좌표를 카메라 좌표계로 변환
    x_cam = x_u * z_m
    y_cam = y_u * z_m
    z_cam = z_m
    
    outputs = []
    outputs.append(
        {"pixel_xy": [x_u, y_u],
        "depth_m": z_m,
        "cam_xyz": np.round([x_cam, y_cam, z_cam], 3).tolist(),
        "undistroted_xyz": np.round([x_u, y_u, z_cam], 3).tolist()}
    )
    # 결과 출력
    print(f"\n📊 클릭한 점의 카메라 좌표:")
    print(f"   픽셀 좌표: ({x}, {y})")
    print(f"   왜곡보정 픽셀: ({x_u:.2f}, {y_u:.2f})")
    print(f"   깊이: {z_m:.3f} m ({z_mm:.1f} mm)")
    print(f"   카메라 XYZ: ({x_cam:.3f}, {y_cam:.3f}, {z_cam:.3f}) m")
    
    return outputs
    

def get_clicked_point_cam_xyz(c_path: str, d_path: str, camera_matrix: np.ndarray, 
                             dist_coeffs: np.ndarray, depth_scale: float = 0.001) -> Optional[List[float]]:
    """
    이미지에서 클릭한 점의 카메라 좌표계 XYZ를 구하는 함수
    
    Args:
        c_path (str): 컬러 이미지 파일 경로
        d_path (str): 깊이 이미지 파일 경로 (.npy)
        camera_matrix (np.ndarray): 카메라 내부 파라미터 행렬 (3x3)
        dist_coeffs (np.ndarray): 왜곡 계수
        depth_scale (float): 깊이 스케일 (기본값: 0.001, mm -> m 변환)
    
    Returns:
        Optional[List[float]]: 카메라 좌표계 XYZ [x, y, z] (미터 단위), 클릭하지 않으면 None
    """
    
    # 이미지와 깊이 데이터 로드
    color_img = cv2.imread(c_path)
    if color_img is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {c_path}")
    
    depth_raw = np.load(d_path)
    
    # 클릭한 점을 저장할 변수
    clicked_point = None
    
    def mouse_callback(event: int, x: int, y: int, flags: int, param) -> None:
        """마우스 클릭 콜백 함수"""
        nonlocal clicked_point
        if event == cv2.EVENT_LBUTTONDOWN:
            clicked_point = (x, y)
            print(f"클릭한 점: ({x}, {y})")
    
    # 윈도우 생성 및 마우스 콜백 설정
    window_name = "Click to get camera XYZ"
    cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)
    cv2.setMouseCallback(window_name, mouse_callback)
    
    # 이미지 표시
    cv2.imshow(window_name, color_img)
    print("이미지에서 원하는 점을 클릭하세요. ESC를 누르면 종료됩니다.")
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # ESC 키로 종료
        if key == 27:
            break
        
        # 클릭한 점이 있으면 처리
        if clicked_point is not None:
            x, y = clicked_point
            outputs = get_cam_xyz(x, y, depth_raw, camera_matrix, dist_coeffs, depth_scale)
            cv2.destroyAllWindows()
            return outputs
    
    cv2.destroyAllWindows()
    return None

def get_cam_xyz_by_input(c_path: str, d_path: str, camera_matrix: np.ndarray, 
                             dist_coeffs: np.ndarray, depth_scale: float = 0.001) -> Optional[List[float]]:
    
    color_img = cv2.imread(c_path)
    depth_raw = np.load(d_path)
    
    x = int(input("Enter x: "))
    y = int(input("Enter y: "))
    
    outputs = get_cam_xyz(x, y, depth_raw, camera_matrix, dist_coeffs, depth_scale)
    return outputs

In [95]:
intr_path2 = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand11/camera_intrinsics_result.json"
camera_matrix2, dist_coeffs2 = load_intrinsics(intr_path2)

intr_path3 = "/home/ros/Eye-to-Hand/data/camera_calibration3/camera_params.json"
camera_matrix3, dist_coeffs3 = load_intrinsics(intr_path3)

outputs = get_clicked_point_cam_xyz(c_path, d_path, camera_matrix, dist_coeffs,0.001)
# outputs2 = get_cam_xyz_by_input(c_path, d_path, camera_matrix2, dist_coeffs2,0.001)
# outputs3 = get_cam_xyz_by_input(c_path, d_path, camera_matrix3, dist_coeffs3,0.001)
outputs3 = get_cam_xyz_by_input(c_path, d_path, camera_matrix, dist_coeffs,0.0005)
# outputs4 = get_cam_xyz_by_input(c_path, d_path, camera_matrix, dist_coeffs,0.00085)
# outputs5 = get_cam_xyz_by_input(c_path, d_path, camera_matrix, dist_coeffs,0.0008)

이미지에서 원하는 점을 클릭하세요. ESC를 누르면 종료됩니다.
클릭한 점: (656, 79)

📊 클릭한 점의 카메라 좌표:
   픽셀 좌표: (656, 79)
   왜곡보정 픽셀: (0.54, -0.40)
   깊이: 0.554 m (554.0 mm)
   카메라 XYZ: (0.301, -0.220, 0.554) m

📊 클릭한 점의 카메라 좌표:
   픽셀 좌표: (656, 79)
   왜곡보정 픽셀: (0.54, -0.40)
   깊이: 0.277 m (554.0 mm)
   카메라 XYZ: (0.151, -0.110, 0.277) m


# Eye-in-Hand

In [15]:
# ▶ 오일러각 → 회전행렬
def euler_to_rotation_matrix(rx, ry, rz):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rx), -np.sin(rx)],
        [0, np.sin(rx), np.cos(rx)]
    ])
    Ry = np.array([
        [np.cos(ry), 0, np.sin(ry)],
        [0, 1, 0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz), np.cos(rz), 0],
        [0, 0, 1]
    ])
    return Rz @ Ry @ Rx

In [57]:
cam2ee_path = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand/cam2ee.json"
cam2ee_path2 = "/home/ros/llm_robot/notebooks/optimized_cam2ee_RANSAC.json"
with open(cam2ee_path2, "r") as f:
    data = json.load(f)

R_cam2ee = np.array(data["R_cam2ee"])
R_ee2cam = np.linalg.inv(R_cam2ee)
t_cam2ee = np.array(data["t_cam2ee"]).reshape(3, 1)

T_cam2ee = np.eye(4)
T_cam2ee[:3, :3] = R_cam2ee
T_cam2ee[:3, 3] = t_cam2ee.flatten()
T_ee2cam = np.linalg.inv(T_cam2ee) 

# 현재 로봇 포즈 가져오기 (coords 사용)
position = np.array(coords[0:3]).reshape(3, 1) / 1000.0  # mm → m
rx, ry, rz = np.radians(coords[3:6])

R_base2ee = euler_to_rotation_matrix(rx, ry, rz)

# --- Base XYZ 계산 ---
# T_base2ee 행렬 구성
T_base2ee = np.eye(4)
T_base2ee[:3, :3] = R_base2ee
T_base2ee[:3, 3] = position.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_xyz_h = np.vstack([cam_xyz, [[1]]])  # 4x1 homogeneous vectr

# base_xyz 계산
base_xyz_h = T_base2ee @ T_cam2ee @ cam_xyz_h
base_xyz = base_xyz_h[:3].flatten()*1000  # 최종 좌표 (단위: m)

R_correction = np.array([
    [0, 1, 0],
    [1, 0, 0],
    [0, 0, -1]
])
base_xyz_corrected = R_correction @ base_xyz
base_xyz_corrected

array([     156.08,       294.6,      524.38])

In [ ]:
with open("/home/ros/llm_robot/notebooks/optimized_cam2ee_icp_point.json", "r") as f:
    data = json.load(f)

R_cam2ee = np.array(data["R_cam2ee"])
t_cam2ee = np.array(data["t_cam2ee"]).reshape(3, 1)

T_cam2ee = np.eye(4)
T_cam2ee[:3, :3] = R_cam2ee
T_cam2ee[:3, 3] = t_cam2ee.flatten()


# ▶ 현재 로봇 포즈 가져오기 (coords 사용)
position = np.array(coords[0:3]).reshape(3, 1) / 1000.0  # mm → m
rx, ry, rz = np.radians(coords[3:6])

R_base2ee = euler_to_rotation_matrix(rx, ry, rz)

# ▶ T_base2ee 행렬 구성
T_base2ee = np.eye(4)
T_base2ee[:3, :3] = R_base2ee
T_base2ee[:3, 3] = position.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_xyz_h = np.vstack([cam_xyz, [[1]]])  # 4x1 homogeneous vectr

# ▶ base_xyz 계산
base_xyz_h = T_base2ee @ T_cam2ee @ cam_xyz_h
base_xyz = base_xyz_h[:3].flatten()*1000  # 최종 좌표 (단위: m)

# ▶ 출력
print(f"카메라 기준 위치: {cam_xyz.flatten()} (m)")
print(f"로봇 base 기준 위치: {base_xyz} (mm)")

카메라 기준 위치: [       0.17      -0.051       0.457] (m)
로봇 base 기준 위치: [     48.646     -270.92      717.04] (mm)


# Eye-to-Hand

In [68]:
MAIN_DIR = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand11"

In [70]:
def cam2base_correction(cam2base_path, outputs):
    with open(cam2base_path, "r") as f:
        data = json.load(f)
        
    R_cam2base = np.array(data["R_cam2base"]).reshape(3, 3)
    t_cam2base = np.array(data["t_cam2base"]).reshape(3, 1)

    T_cam2base = np.eye(4)
    T_cam2base[:3, :3] = R_cam2base
    T_cam2base[:3, 3] = t_cam2base.flatten()
    
    T_camera_to_robot_coord = np.array([
    [ 1,  0,  0],
    [0,  1,  0],
    [ 0, 0,  1]
    ]) 

    # 보정 적용
    R_corrected = T_camera_to_robot_coord @ R_cam2base
    t_corrected = T_camera_to_robot_coord @ t_cam2base

    T_cam2base_corrected = np.eye(4)
    T_cam2base_corrected[:3, :3] = R_corrected
    T_cam2base_corrected[:3, 3] = t_corrected.flatten()

    cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3,1)
    cam_homog = np.append(cam_xyz, 1.0)
    base_homog =   T_cam2base_corrected @ cam_homog
    base_xyz_homog = base_homog[:3]*1000      # X, Y, Z (병진 포함)
    print(f"Base XYZ (Homogeneous): {base_xyz_homog}")

### pure

In [106]:
cam2base_path = f"{MAIN_DIR}/cam2base_icp_point.json"
cam2base_correction(cam2base_path, outputs)

cam2base_path2 = f"{MAIN_DIR}/cam2base_table_normal_fix.json"
cam2base_correction(cam2base_path2, outputs)
# cam2base_correction(cam2base_path, outputs2)
# cam2base_correction(cam2base_path, outputs3)

cam2base_path3 = f"{MAIN_DIR}/cam2base_calibrated.json"
cam2base_correction(cam2base_path3, outputs)

# cam2base_path4 = f"{MAIN_DIR}/cam2base_table_normal_fix_ransac2.json"
# cam2base_correction(cam2base_path4, outputs)

cam2base_path5 = f"../data/Calibration/Eye-to-Hand12/cam2base.json"
cam2base_correction(cam2base_path5, outputs)

cam2base_path6 = f"../data/Calibration/Eye-to-Hand12/cam2base_icp_point.json"
cam2base_correction(cam2base_path6, outputs)


Base XYZ (Homogeneous): [    -78.233      242.26       24.05]
Base XYZ (Homogeneous): [    -97.643      145.99     -31.751]
Base XYZ (Homogeneous): [    -88.533      242.35       21.79]
Base XYZ (Homogeneous): [     335.42      113.61     -16.201]
Base XYZ (Homogeneous): [     342.61      117.74     -9.2285]


In [12]:
cam2base_path2 = f"{MAIN_DIR}/cam2base_icp.json"
cam2base_correction(cam2base_path2, outputs)
cam2base_correction(cam2base_path2, outputs2)
cam2base_correction(cam2base_path2, outputs3)

Base XYZ (Homogeneous): [     97.511     -43.467     -15.392]
Base XYZ (Homogeneous): [     98.526      -45.42     -15.786]
Base XYZ (Homogeneous): [     107.48     -39.441     -14.745]


In [30]:

# cam2base_correction(cam2base_path, outputs3)
# cam2base_correction(cam2base_path, outputs4)
# cam2base_correction(cam2base_path, outputs5)

In [39]:
cam2base_path = f"{MAIN_DIR}/cam2base_icp_point_refined.json"
cam2base_correction(cam2base_path, outputs)
cam2base_correction(cam2base_path, outputs2)
cam2base_correction(cam2base_path, outputs3)
cam2base_correction(cam2base_path, outputs4)
cam2base_correction(cam2base_path, outputs5)

Base XYZ (Homogeneous): [     88.931     -77.429       -24.7]
Base XYZ (Homogeneous): [     94.053     -81.715      3.2175]
Base XYZ (Homogeneous): [     95.202     -84.941      30.093]
Base XYZ (Homogeneous): [     97.326     -89.242      57.913]
Base XYZ (Homogeneous): [     98.475     -92.468      84.788]


### ICP

In [48]:
cam2base_path = f"{MAIN_DIR}/cam2base_icp.json"
with open(cam2base_path, "r") as f:
    data = json.load(f)

R_cam2base = np.array(data["R_cam2base"])
t_cam2base = np.array(data["t_cam2base"]).reshape(3, 1)

T_cam2base = np.eye(4)
T_cam2base[:3, :3] = R_cam2base
T_cam2base[:3, 3] = t_cam2base.flatten()

T_camera_to_robot_coord = np.array([
    [ 1,  0,  0],
    [0,  1,  0],
    [ 0, 0,  -1]
]) 

# 보정 적용
R_corrected = T_camera_to_robot_coord @ R_cam2base
t_corrected = T_camera_to_robot_coord @ t_cam2base

T_cam2base_corrected = np.eye(4)
T_cam2base_corrected[:3, :3] = R_corrected
T_cam2base_corrected[:3, 3] = t_corrected.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_homog = np.append(cam_xyz, 1.0)
base_homog =   T_cam2base_corrected @ cam_homog2
base_xyz_homog = base_homog[:3]*1000      # X, Y, Z (병진 포함)
print(f"Base XYZ (Homogeneous): {base_xyz_homog}")

Base XYZ (Homogeneous): [    -86.776      235.61     -24.686]


In [47]:
cam2base_path = f"{MAIN_DIR}/cam2base_icp_point.json"
with open(cam2base_path, "r") as f:
    data = json.load(f)

R_cam2base = np.array(data["R_cam2base"])
t_cam2base = np.array(data["t_cam2base"]).reshape(3, 1)

T_cam2base = np.eye(4)
T_cam2base[:3, :3] = R_cam2base
T_cam2base[:3, 3] = t_cam2base.flatten()

T_camera_to_robot_coord = np.array([
    [ 1,  0,  0],
    [0,  1,  0],
    [ 0, 0,  -1]
]) 

# 보정 적용
R_corrected = T_camera_to_robot_coord @ R_cam2base
t_corrected = T_camera_to_robot_coord @ t_cam2base

T_cam2base_corrected = np.eye(4)
T_cam2base_corrected[:3, :3] = R_corrected
T_cam2base_corrected[:3, 3] = t_corrected.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_homog = np.append(cam_xyz, 1.0)
base_homog =   T_cam2base_corrected @ cam_homog
base_xyz_homog = base_homog[:3]*1000      # X, Y, Z (병진 포함)
print(f"Base XYZ (Homogeneous): {base_xyz_homog}")

Base XYZ (Homogeneous): [     203.16     -28.635     -61.315]


In [46]:
cam2base_path = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand5/cam2base_icp_point1.json"
with open(cam2base_path, "r") as f:
    data = json.load(f)

R_cam2base = np.array(data["R_cam2base"])
t_cam2base = np.array(data["t_cam2base"]).reshape(3, 1)

T_cam2base = np.eye(4)
T_cam2base[:3, :3] = R_cam2base
T_cam2base[:3, 3] = t_cam2base.flatten()

T_camera_to_robot_coord = np.array([
    [ 1,  0,  0],
    [0,  1,  0],
    [ 0, 0,  -1]
]) 

# 보정 적용
R_corrected = T_camera_to_robot_coord @ R_cam2base
t_corrected = T_camera_to_robot_coord @ t_cam2base

T_cam2base_corrected = np.eye(4)
T_cam2base_corrected[:3, :3] = R_corrected
T_cam2base_corrected[:3, 3] = t_corrected.flatten()

cam_xyz = np.array(outputs[0]["cam_xyz"]).reshape(3, 1)
cam_homog = np.append(cam_xyz, 1.0)
base_homog =   T_cam2base_corrected @ cam_homog
base_xyz_homog = base_homog[:3]*1000      # X, Y, Z (병진 포함)
print(f"Base XYZ (Homogeneous): {base_xyz_homog}")

Base XYZ (Homogeneous): [      210.1      60.259      -27.35]


In [119]:
import numpy as np
import json
from scipy.spatial.transform import Rotation

def cam2base(cam_xyz, cam2base_path):
    with open(cam2base_path, "r") as f:
        data = json.load(f)

    R_cam2base = np.array(data["R_cam2base"]).reshape(3,3)
    t_cam2base = np.array(data["t_cam2base"]).reshape(3, 1)

    cam_xyz = np.array(cam_xyz).reshape(3, 1)
    cam_xyz_h = np.vstack([cam_xyz, [[1]]])  # 4x1 homogeneous vectr
    
    T_cam2base = np.eye(4)
    T_cam2base[:3, :3] = R_cam2base
    T_cam2base[:3, 3] = t_cam2base.flatten()

    # base_xyz 계산
    base_xyz_h = T_cam2base @ cam_xyz_h
    base_xyz = base_xyz_h[:3].flatten()*1000  # 최종 좌표 (단위: m)
    
    return base_xyz


In [124]:
outputs[0]["cam_xyz"]

[np.float64(0.0607), np.float64(0.0354), np.float64(0.517)]

In [125]:
cam2base(outputs[0]["cam_xyz"], cam2base_path)

array([     197.43,      52.929,     -7.5923])